In [14]:
from core import vector_store, embed_model, llm

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=10)
retriever = index.as_retriever(similarity_top_k=10)

In [15]:
response = retriever.retrieve('сколько прекрасных витязей выходят из вод?')

response

[NodeWithScore(node=TextNode(id_='bb417b1f-804b-4c62-90f1-5bd697427877', embedding=None, metadata={'file_path': '/Users/akscheglov/hackaton/elastic/../data/texts/oop.md', 'file_name': 'oop.md', 'file_type': 'text/markdown', 'file_size': 48646, 'creation_date': '2024-12-30', 'last_modified_date': '2024-12-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6241e81e-6448-4e5e-b262-40bc1c50266d', node_type='4', metadata={'file_path': '/Users/akscheglov/hackaton/elastic/../data/texts/oop.md', 'file_name': 'oop.md', 'file_type': 'text/markdown', 'file_size': 48646, 'creation_date': '2024-12-30', 'last_modified_date': '2024-12-30'}, hash='d8ec583e2c63508794129bf274ba08fae29abdd09a64c09665b623bc897ba750'), <Node

In [ ]:
answer = query_engine.query('сколько богатырей выходят из моря?')

answer